<a href="https://colab.research.google.com/github/mlluizza/AIRBNB-RIO-DE-JANEIRO/blob/main/AIRBNB_RIO_DE_JANEIRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Airbnb Rio - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns**

# **Contexto**

No Airbnb, qualquer pessoa que tenha um quarto um imóvel de qualquer tipo (apartamento, casa, chalé, pousada, etc.) pode ofertar o seu imóvel para ser alugado por diária.

Você cria o seu perfil de host (pessoa que disponibiliza um imóvel para aluguel por diária) e cria o anúncio do seu imóvel.

Nesse anúncio, o host deve descrever as características do imóvel da forma mais completa possível, de forma a ajudar os locadores/viajantes a escolherem o melhor imóvel para eles(e de forma a tornar o seu anúncio mais atrativo)

Existem dezenas de personalizações possíveis no seu anúncio,desde quantidade mínima de diária, preço, quantidade de quartos, até regras de cancelamento, taxa extra para hóspedes exras, exigências de verificaçã de identidade do locador, etc.


# **Objetivo**

Construir um modelo de previsão de preço que permita um pessoa comum que possui um imóvel possa saber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesma características) ou não.

**Importar Biblioteca e Bases de Dados**

In [ ]:
import pandas as pd
import pathlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor


In [ ]:
meses = {'jan': 1, 'fev':2, 'mar':3, 'abr':4, 'mai':5,'jun':6, 'jul':7, 'ago':8,'set':9, 'out':10,'nov':11, 'dez':12}

from google.colab import drive
drive.mount('/content/gdrive')
caminho_bases = pathlib.Path('/content/gdrive/MyDrive/dataset')

base_airbnb = pd.DataFrame()

for arquivo in caminho_bases.iterdir():

  if arquivo.is_file() and arquivo.suffix == '.csv':
    nome_mes = arquivo.name[:3]
    if nome_mes in meses:
      mes = meses[nome_mes]

      ano = arquivo.name[-8:]
      ano = int(ano.replace('.csv', ''))

      df = pd.read_csv(caminho_bases / arquivo.name)
      df['ano'] = ano
      df['mes'] = mes

      base_airbnb = pd.concat([base_airbnb, df], ignore_index=True)

display(base_airbnb)

*   Temos muitas colunas,  o modelo pode acabar ficando muito lento.
*   Além disso uma análise rápida permite ver que várias colunas não são necessárias para o modelo de previsão, por isso irei excluir algumas colunas da nossa base.

*   Tipo de colunas que irei excluir:
1. IDs, Links e informações não relevantes para o modelo   
2. Colunas repetidas ou extremamente parecidas com outra (que dão a mesma informação para o modelo. Ex: Data x Ano/Mês)
3. Colunas preenchidas com texto livre ->  Não rodarei nenhuma análise de palavras ou algo do tipo
4. Colunas em que todos ou quase todos os valores são iguais.

*   Para isso, irei criar um arquivo em excel com 1.000 primeiros  registros e fazer uma análise qualitativa








In [ ]:
from os import sep
print(list(base_airbnb.columns))
base_airbnb.head(1000).to_csv('primeiros_registros.csv', sep=';')

In [ ]:
from google.colab import files
files.download('primeiros_registros.csv')

**Depois da análise qualitativa das colunas, levando em conta os critérios explicativos acima, ficamos com as seguintes colunas:**

In [ ]:
colunas = ['host_response_time','host_response_rate','host_is_superhost','host_listings_count','latitude','longitude','property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type','amenities','price','security_deposit','cleaning_fee','guests_included','extra_people','minimum_nights','maximum_nights','number_of_reviews','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','instant_bookable','is_business_travel_ready','cancellation_policy','ano','mes']

base_airbnb = base_airbnb.loc[:, colunas]
print(list(base_airbnb.columns))
display(base_airbnb)

### **Tratar valores faltantes**


*   Visualizando os dados, percebemos que existe uma grande disparidade em dados faltantes. As colunas com mais de de 300.00 valores NaN foram excluíddas da análise
*   Para as outras colunas, como temos muitos dados (mais de 900.000 linhas) excluir as linhas que contém dados NaN.

In [ ]:
for coluna in base_airbnb:
  if base_airbnb[coluna].isnull().sum() > 300000:
    base_airbnb = base_airbnb.drop(coluna,axis=1)
print(base_airbnb.isnull().sum())

In [ ]:
base_airbnb = base_airbnb.dropna()

print(base_airbnb.shape)
print(base_airbnb.isnull().sum())

### **Verificar Tipos de dados em cada coluna**

In [ ]:
print(base_airbnb.dtypes)
print('-'*60)
print(base_airbnb.iloc[0])

**Como preço e extra people estão reconhecidos como objeto (ao invés de ser um float) temos que mudar o tipo de variável da coluna.**

In [ ]:
# price
base_airbnb['price'] = base_airbnb['price'].str.replace('$', '')
base_airbnb['price'] = base_airbnb['price'].str.replace(',', '')
base_airbnb['price'] = base_airbnb['price'].astype(np.float32, copy=False)

# extra people
base_airbnb['extra_people'] = base_airbnb['extra_people'].str.replace('$', '')
base_airbnb['extra_people'] = base_airbnb['extra_people'].str.replace(',', '')
base_airbnb['extra_people'] = base_airbnb['extra_people'].astype(np.float32, copy=False)

#Verificando os tipos
print(base_airbnb.dtypes)

## **ANÁLISE EXPLORATÓRIA E TRATAR OUTLIERS**

In [ ]:
temp_base_airbnb_for_corr = base_airbnb.copy()


temp_base_airbnb_for_corr['host_is_superhost'] = temp_base_airbnb_for_corr['host_is_superhost'].map({'t': 1, 'f': 0})
temp_base_airbnb_for_corr['instant_bookable'] = temp_base_airbnb_for_corr['instant_bookable'].map({'t': 1, 'f': 0})
temp_base_airbnb_for_corr['is_business_travel_ready'] = temp_base_airbnb_for_corr['is_business_travel_ready'].map({'t': 1, 'f': 0})


numeric_base_airbnb = temp_base_airbnb_for_corr.select_dtypes(include=[np.number])

plt.figure(figsize=(15, 10))
sns.heatmap(numeric_base_airbnb.corr(), annot=True, cmap="Greens")

In [ ]:
base_airbnb['host_is_superhost'] = base_airbnb['host_is_superhost'].map({'t': 1, 'f': 0})
base_airbnb['instant_bookable'] = base_airbnb['instant_bookable'].map({'t': 1, 'f': 0})
base_airbnb['is_business_travel_ready'] = base_airbnb['is_business_travel_ready'].map({'t': 1, 'f': 0})

numeric_base_airbnb = base_airbnb.select_dtypes(include=[np.number])
print(numeric_base_airbnb.corr())

### **Definição de Funções para Análise de Outliers**

Definir algumas funções para ajudar na análise de outliers das colunas

In [ ]:
def limites(coluna):
  q1 = coluna.quantile(0.25)
  q3 = coluna.quantile(0.75)
  amplitude = q3 - q1
  return q1 - 1.5 * amplitude, q3 + 1.5 * amplitude
def excluir_outliers(df,nome_coluna):
  qtde_linhas = df.shape[0]
  lim_inf, lim_sup = limites(df[nome_coluna])
  df = df.loc[(df[nome_coluna] >= lim_inf) & (df[nome_coluna] <= lim_sup), :]
  linhas_removidas = qtde_linhas - df.shape[0]
  return df, linhas_removidas

In [ ]:
def diagrama_caixa(coluna):
  fig, (ax1, ax2) = plt.subplots(1,2)
  fig.set_size_inches(15, 5)
  sns.boxplot(x=coluna, ax=ax1)
  ax2.set_xlim(limites(coluna))
  sns.boxplot(x=coluna, ax=ax2)

def histograma(coluna):
  plt.figure(figsize=(15, 5))
  sns.distplot(coluna,  hist=True )

def grafico_barra(coluna):
  plt.figure(figsize=(15, 5))
  ax = sns.barplot(x=coluna.value_counts().index, y=coluna.value_counts())
  ax.set_xlim(limites(coluna))

### **PRICE**

In [ ]:
diagrama_caixa(base_airbnb['price'])
histograma(base_airbnb['price'])

Como estou construindo um modelo para imóveis comuns, acredito que os valores acima do limite superior serão apartamentos de altíssimo luxo, que não é o nosso objetivo. Por isso, podemos excluir esses outliers,

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'price')
print('{} linhas removidas'.format(linhas_removidas))

In [ ]:
histograma(base_airbnb['price'])
print(base_airbnb.shape)

###**EXTRA_PEOPLE**

In [ ]:
diagrama_caixa(base_airbnb['extra_people'])
histograma(base_airbnb['extra_people'])

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'extra_people')
print('{} linhas removidas'.format(linhas_removidas))

In [ ]:
histograma(base_airbnb['extra_people'])
print(base_airbnb.shape)

##**host_listings_count**

In [ ]:
diagrama_caixa(base_airbnb['host_listings_count'])
grafico_barra(base_airbnb['host_listings_count'])

Podemos excluir os outliers, porque para o objetivo do nosso projeto porque hosts com mais de 6 imóveis no airbnb não é o público alvo do objetivo do projeto (imagino que sejam imobiliários ou profissionais que gereciam imóveis no airbnb)

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'host_listings_count')
print('{} linhas removidas'.format(linhas_removidas))

# **accommodates**

In [ ]:
diagrama_caixa(base_airbnb['accommodates'])
grafico_barra(base_airbnb['accommodates'])

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'accommodates')
print('{} linhas removidas'.format(linhas_removidas))

# **bathrooms**

In [ ]:
diagrama_caixa(base_airbnb['bathrooms'])
plt.figure(figsize=(15, 5))
sns.barplot(x=base_airbnb['bathrooms'].value_counts().index, y=base_airbnb['bathrooms'].value_counts())

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'bathrooms')
print('{} linhas removidas'.format(linhas_removidas))

# **bedrooms**

In [ ]:
diagrama_caixa(base_airbnb['bedrooms'])
grafico_barra(base_airbnb['bedrooms'])

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'bedrooms')
print('{} linhas removidas'.format(linhas_removidas))

# **beds**

In [ ]:
diagrama_caixa(base_airbnb['beds'])
grafico_barra(base_airbnb['beds'])

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'beds')
print('{} linhas removidas'.format(linhas_removidas))

# **guests_included**

In [ ]:
#diagrama_caixa(base_airbnb['guests_included'])
#grafico_barra(base_airbnb['guests_included'])
print(limites(base_airbnb['guests_included']))
plt.figure(figsize=(15, 5))
sns.barplot(x=base_airbnb['guests_included'].value_counts().index, y=base_airbnb['guests_included'].value_counts())

Remoção dessa feature da análise. Parece que os usuários do airbnb usam muito o valor padrão do airbnb como guest included. Isso pode levar o nosso modelo a considerar uma feature que na verdade não é essencial para definição do preço, por isso, me parece melhor excluir a coluna da análise

In [ ]:
base_airbnb = base_airbnb.drop('guests_included', axis=1)
base_airbnb.shape

# **minimum_nights**

In [ ]:
diagrama_caixa(base_airbnb['minimum_nights'])
grafico_barra(base_airbnb['minimum_nights'])

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'minimum_nights')
print('{} linhas removidas'.format(linhas_removidas))

# **maximum_nights**

In [ ]:
diagrama_caixa(base_airbnb['maximum_nights'])
grafico_barra(base_airbnb['maximum_nights'])

In [ ]:
base_airbnb = base_airbnb.drop('maximum_nights', axis=1)
base_airbnb.shape

# **number_of_reviews**        


In [ ]:
diagrama_caixa(base_airbnb['number_of_reviews'])
grafico_barra(base_airbnb['number_of_reviews'])

In [ ]:
base_airbnb = base_airbnb.drop('number_of_reviews', axis=1)
base_airbnb.shape

### **Tratamento de Colunas de Valores de Texto**


*   **property_type**


In [ ]:
print(base_airbnb['property_type'].value_counts())

plt.figure(figsize=(15, 5))
grafico = sns.countplot(x='property_type', data=base_airbnb)
grafico.tick_params(axis='x', rotation=90)

In [ ]:
tabela_tipos_casa = base_airbnb['property_type'].value_counts()
colunas_agrupar = []

for tipo in tabela_tipos_casa.index:
  if tabela_tipos_casa[tipo] < 2000:
    colunas_agrupar.append(tipo)
print(colunas_agrupar)

for tipo in colunas_agrupar:
  base_airbnb.loc[base_airbnb['property_type'] == tipo, 'property_type'] = 'Outros'

print(base_airbnb['property_type'].value_counts())

In [ ]:
plt.figure(figsize=(15, 5))
grafico = sns.countplot(x='property_type', data=base_airbnb)
grafico.tick_params(axis='x', rotation=90)

*   **room_type**

In [ ]:
print(base_airbnb['room_type'].value_counts())

plt.figure(figsize=(15, 5))
grafico = sns.countplot(x='room_type', data=base_airbnb)
grafico.tick_params(axis='x', rotation=90)

*   **bed_type**

In [ ]:
print(base_airbnb['bed_type'].value_counts())

plt.figure(figsize=(15, 5))
grafico = sns.countplot(x='bed_type', data=base_airbnb)
grafico.tick_params(axis='x', rotation=90)

*   **cancellation_policy**

In [ ]:
print(base_airbnb['cancellation_policy'].value_counts())

plt.figure(figsize=(15, 5))
grafico = sns.countplot(x='cancellation_policy', data=base_airbnb)
grafico.tick_params(axis='x', rotation=90)

# Agrupando categorias de cancellation_policy
tabela_cancellation = base_airbnb['cancellation_policy'].value_counts()
colunas_agrupar = []

for tipo in tabela_cancellation.index:
  if tabela_cancellation[tipo] < 10000:
    colunas_agrupar.append(tipo)
print(colunas_agrupar)

for tipo in colunas_agrupar:
  base_airbnb.loc[base_airbnb['cancellation_policy'] == tipo, 'cancellation_policy'] = 'Outros'

print(base_airbnb['cancellation_policy'].value_counts())
plt.figure(figsize=(15, 5))
grafico = sns.countplot(x='cancellation_policy', data=base_airbnb)
grafico.tick_params(axis='x', rotation=90)

*   **room_type**

In [ ]:
print(base_airbnb['room_type'].value_counts())

plt.figure(figsize=(15, 5))
grafico = sns.countplot(x='room_type', data=base_airbnb)
grafico.tick_params(axis='x', rotation=90)

*   **bed_type**

In [ ]:
print(base_airbnb['bed_type'].value_counts())

plt.figure(figsize=(15, 5))
grafico = sns.countplot(x='bed_type', data=base_airbnb)
grafico.tick_params(axis='x', rotation=90)

*   **amenities**

Como temos uma diversidade muito grande de amenities e, às vezes, as mesmas amenities podem ser escritas de forma diferente, vamos avaliar a quantidade de amenities como parâmetro para o nosso modelo.

In [ ]:
print(base_airbnb['amenities'].iloc[0].split(','))
print(len(base_airbnb['amenities'].iloc[0].split(',')))

base_airbnb['n_amenities'] = base_airbnb['amenities'].str.split(',').apply(len)

In [ ]:
base_airbnb = base_airbnb.drop('amenities', axis=1)
base_airbnb.shape

In [ ]:
diagrama_caixa(base_airbnb['n_amenities'])
grafico_barra(base_airbnb['n_amenities'])

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'n_amenities')
print('{} linhas removidas'.format(linhas_removidas))

### **Encoding**

Ajustar as features para facilitar o trabalho do modelo futuro (features de categoria, true e false,etc.)



*   Features de Valores True ou False, irei substituir True por 1 e False por 0.
*   Feature de Categoria (feature em que os valores da coluna são textos) utilizando o método de encoding de variáveis dummies.



In [ ]:
colunas_tf = ['host_is_superhost', 'instant_bookable', 'is_business_travel_ready']
base_airbnb_cod = base_airbnb.copy()
for coluna in colunas_tf:
  base_airbnb_cod.loc[base_airbnb_cod[coluna] == 't', coluna] = 1
  base_airbnb_cod.loc[base_airbnb_cod[coluna] == 'f', coluna] = 0

In [ ]:
colunas_categorias = ['property_type', 'room_type', 'bed_type', 'cancellation_policy']
base_airbnb_cod = pd.get_dummies(base_airbnb_cod, columns=colunas_categorias)
display(base_airbnb_cod.head())


# **Visualização de Mapa das Propriedades**

In [ ]:
import urllib.request
from PIL import Image
import io
import http.client


min_longitude = base_airbnb['longitude'].min()
max_longitude = base_airbnb['longitude'].max()
min_latitude = base_airbnb['latitude'].min()
max_latitude = base_airbnb['latitude'].max()
box = (min_longitude, max_longitude, min_latitude, max_latitude)


fig, ax= plt.subplots()
fig.set_size_inches(16,12)

image_loaded = False
try:
    with urllib.request.urlopen('https://i.ibb.co/52dDkxT/map-8.png') as url:
        f = io.BytesIO(url.read())
        a = np.array(Image.open(f))
    ax.imshow(a,zorder=0, extent=(box[0],-43.1041,-23.082,box[3]), aspect='equal')
    image_loaded = True
except (urllib.error.URLError, IOError, Image.UnidentifiedImageError, http.client.IncompleteRead) as e:
    print(f"Warning: Could not load background image due to: {e}. Plotting scatter points without background image.")

ax.set_title('Listings location and info about the city')
ax.set_xlim(box[0],box[1])
ax.set_ylim(box[2],box[3])#-23.0729
ax.annotate('offset values',
            xy=(-43.18, -22.82),
            xytext=(-43.2,-22.872), backgroundcolor='yellow',bbox=dict(facecolor='#ffa500', alpha=0.5, edgecolor='red', joinstyle='round'),
            arrowprops=dict(headwidth=8, width=1, color='#ffa500', connectionstyle="arc3, rad=0.3"),
            fontsize=12)
ax.annotate('Parque Olímpico',
            xy=(-43.39, -22.98),
            xytext=(-43.48,-23.055), backgroundcolor='yellow',bbox=dict(facecolor='#ffa500', alpha=0.5, edgecolor='red', joinstyle='round'),
            arrowprops=dict(headwidth=8, width=1, color='#ffa500', connectionstyle="arc3, rad=-0.2"),
            fontsize=12)
ax.annotate('City center',
            xy=(-43.163, -22.91),
            xytext=(-43.165,-23.02), backgroundcolor='yellow',bbox=dict(facecolor='#ffa500', alpha=0.5, edgecolor='red', joinstyle='round'),
            arrowprops=dict(headwidth=8, width=1, color='#ffa500', connectionstyle="arc3, rad=0.5"),
            fontsize=12)
ax.annotate('Touristic spots',
            xy=(-43.19, -22.995),
            xytext=(-43.32,-23.055), backgroundcolor='yellow',bbox=dict(facecolor='#ffa500', alpha=0.5, edgecolor='red', joinstyle='round'),
            arrowprops=dict(headwidth=8, width=1, color='#ffa500', connectionstyle="arc3, rad=0.3"),
            fontsize=12)
_ = sns.scatterplot(x='longitude', y='latitude', data=base_airbnb, ax=ax, zorder=1,color='black',alpha=0.6, s=0.1,edgecolor=None)

# **Modelo de Previsão**



*   Métricas de Avaliação


In [ ]:
def avaliar_modelo(nome_modelo, y_teste, previsao):
  r2 = r2_score(y_teste, previsao)
  rmse = np.sqrt(mean_squared_error(y_teste, previsao))
  return f'Modelo {nome_modelo}:\nR²: {r2}\nRMSE: {rmse}'



*   Escolha dos Modelos a Serem Testados


1.   RandomForest
2.   LinearRegression
3.   ExtraTree




In [ ]:
modelo_rf = RandomForestRegressor()
modelo_lr = LinearRegression()
modelo_et = ExtraTreesRegressor()

modelos = { 'RandomForest': modelo_rf,
           'LinearRegression': modelo_lr,
           'ExtraTree': modelo_et,
           }

y = base_airbnb['price']
X = base_airbnb.drop('price', axis=1)



*   Separa os dados em treino e teste + Treino do Modelo



In [ ]:
from sklearn.model_selection import train_test_split

y = base_airbnb_cod['price']
X = base_airbnb_cod.drop('price', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

for nome_modelo, modelo in modelos.items():
  #treinar
  modelo.fit(X_train, y_train)
  #testar
  previsao= modelo.predict(X_test)
  print(avaliar_modelo(nome_modelo, y_test, previsao))

### **Análise do Melhor Modelo**

In [ ]:
for nome_modelo, modelo in modelos.items():
    #testar
    previsao = modelo.predict(X_test)
    print(avaliar_modelo(nome_modelo, y_test, previsao))

- Modelo Escolhido como Melhor Modelo: ExtraTressRegressor

    Esse foi o modelo com maior valor de R² e ao mesmo tempo o menor valor de RSME. Como não tivemos uma grande diferença de velocidade de treino e de previsão desse modelo com o modelo de RandomForest (que teve resultados próximos de R² e RSME), vamos escolher o Modelo ExtraTrees.
    
    O modelo de regressão linear não obteve um resultado satisfatório, com valores de R² e RSME muito piores do que os outros 2 modelos.
    
- Resultados das Métricas de Avaliaçõ no Modelo Vencedor:<br>
Modelo ExtraTrees:<br>
R²:97.49%<br>
RSME:41.99

### **Ajustes e Melhorias no Melhor Modelo**

In [ ]:
#print(modelo_et.feature_importances_)
#print(X_train.columns)
importancia_features = pd.DataFrame(modelo_et.feature_importances_, X_train.columns)
importancia_features = importancia_features.sort_values(by=0, ascending=False)
display(importancia_features)
plt.figure(figsize=(15, 5))
ax = sns.barplot(x=importancia_features.index, y=importancia_features[0])
ax.tick_params(axis='x', rotation=90)

### **Ajustes Finais no Modelo**

- is_business_travel ready não parece ter muito impacto no nosso modelo. Por isso, para chegar em um modelo mais simples, excluir essa feature e testar o modelo sem ela.

In [ ]:
base_airbnb_cod = base_airbnb_cod.drop('is_business_travel_ready', axis=1)

y = base_airbnb_cod['price']
X = base_airbnb_cod.drop('price', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

modelo_et.fit(X_train, y_train)
previsao = modelo_et.predict(X_test)
print(avaliar_modelo('ExtraTrees', y_test, previsao))

In [ ]:
base_teste = base_airbnb_cod.copy()
for coluna in base_teste:
    if 'bed_type' in coluna:
        base_teste = base_teste.drop(coluna, axis=1)
print(base_teste.columns)
y = base_teste['price']
X = base_teste.drop('price', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

modelo_et.fit(X_train, y_train)
previsao = modelo_et.predict(X_test)
print(avaliar_modelo('ExtraTrees', y_test, previsao))

In [ ]:
print(previsao)

Através da análise, foram identificados vários insights valiosos:



*   Variáveis como o tipo de imóvel, o número de quartos e a avaliação do anfitrião se mostraram altamente relevantes para a determinação do preço da diária.
*   Apartamentos de luxo e imóveis com características extremas foram identificados como outliers, sendo excluídos para garantir que o modelo se concentrasse em imóveis comuns.
*   A latitude e longitude dos imóveis mostraram uma correlação significativa com o preço, sugerindo que a localização tem grande influência na definição do valor da diária.
*   As avaliações dos hóspedes, como limpeza, precisão e comunicação, desempenham um papel crucial na precificação, com imóveis melhor avaliados tendendo a ter preços mais elevados.
*   A remoção de variáveis menos relevantes, como o tipo de política de cancelamento, ajudou a melhorar a performance do modelo, tornando-o mais eficiente sem perda de precisão.






